In [1]:
#--build and train a basic CNN model for CIFAR10 dataset

#--read and understand dataset
#--convert dataset into tensors -> prepare train/val -> prepare dataloader
#--model -> build -> train -> evaluate -> inference

In [2]:
#--mount gdrive for dataset
from google.colab import drive
drive.mount('/content/mount_gdrive')

Mounted at /content/mount_gdrive


In [26]:
#--import required libraries
import torch
import torch.nn as nn
import torchvision

from torchvision.datasets.utils import download_url
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
import torch.nn.functional as F


In [9]:
#--download cifar10 dataset
data_save_path="/content/mount_gdrive/MyDrive/Learning_AI_dataset/pytorch_dataset"
dataset=CIFAR10(root=data_save_path,
                download=True,
                )

100%|██████████| 170498071/170498071 [00:01<00:00, 100607261.55it/s]


Extracting /content/mount_gdrive/MyDrive/Learning_AI_dataset/pytorch_dataset/cifar-10-python.tar.gz to /content/mount_gdrive/MyDrive/Learning_AI_dataset/pytorch_dataset


In [11]:
#--read and understand dataset
print(len(dataset))
print(dataset[0])

50000
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7FE537D0A890>, 6)


In [14]:
#--convert them to tensors
dataset=CIFAR10(root=data_save_path,
                download=True,
                transform=ToTensor()
                )

dataset[0]

Files already downloaded and verified


(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
          [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
          [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
          ...,
          [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
          [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
          [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],
 
         [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
          [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
          [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
          ...,
          [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
          [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
          [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],
 
         [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
          [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
          [0.0824, 0.0000, 0.0314,  ...,

In [16]:
#--dataset converted into tensors
print(dataset[0][0].shape)

torch.Size([3, 32, 32])


In [19]:
#--prepare train/test split then dataloader
batch_size=128
train_ds,val_ds=random_split(dataset,[40000,10000])

train_loader=DataLoader(train_ds,batch_size,shuffle=True)
val_loader=DataLoader(val_ds,batch_size)

In [22]:
#--build basic CNN
basic_cnn=nn.Sequential(
    nn.Conv2d(3,8,kernel_size=3,stride=1,padding=1),
    nn.MaxPool2d(2,2)
)

In [23]:
basic_cnn

Sequential(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [25]:
for images, labels in train_loader:
  print("image shape :",images.shape)
  preds=basic_cnn(images)
  print("output shape :",preds.shape)
  break

image shape : torch.Size([128, 3, 32, 32])
output shape : torch.Size([128, 8, 16, 16])


In [27]:
#--define accuracy function
def acc(preds,labels):
  '''
  preds: softmax predicted prob
  labels: onehot ground truth labels
  output: accuracy
  '''
  max_prob,max_prob_indices=torch.max(preds,dim=1)
  return torch.tensor(
     torch.sum(max_prob_indices==labels).item() /len(preds)
  )

In [30]:
#---base model helper functions
class ImageClassificationBase(nn.Module):
  def training_step(self,batch): #--perform model training on each batch
    images,labels=batch
    preds=self(images)
    loss=F.softmax(preds,labels)
    return loss

  def validation_step(self,batch): #--perform model inference on each batch
    images,labels=batch
    preds=self(images)
    val_loss=F.softmax(preds,labels)
    val_acc=acc(preds,labels)
    return {
        "val_loss":val_loss,
        "val_acc":val_acc
    }

  def validation_epoch_end(self,outputs): #--accumulate val loss and acc and compute average for all the batches
    batch_loss=[x['val_loss'] for x in outputs]
    batch_acc=[x['val_acc'] for x in outputs]

    avg_loss=torch.stack(batch_loss).mean()
    avg_acc=torch.stack(batch_acc).mean()

    return {
        "val_loss":avg_loss.item(),
        "val_acc":avg_acc.item()
    }

    def epoch_end(self,epoch,result):
      print("Epoch :[{}], val_loss : [{:.4f}], val_acc : [{:.4f}]".format(epoch,result['val_loss'],result['val_acc']))


In [38]:
#--define CIFAR class model
class Cifar10_cnn_model(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.network=nn.Sequential(

        #--1st block
        nn.Conv2d(3,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2), #--> output shape -> [batch_size,64,16,16]

        #--2nd block
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2), #--> output shape -> [batch_size,128,8,8]

        #--3rd block
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2), #--> output shape -> [batch_size,256,4,4]

        #--4th block
        nn.Flatten(),
        nn.Linear(256*4*4,1024),
        nn.ReLU(),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,xb):

      return self.network(xb)

In [39]:
model=Cifar10_cnn_model()
model

Cifar10_cnn_model(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=4096, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_fe

In [40]:
model.parameters()

<generator object Module.parameters at 0x7fe5374e9fc0>

In [41]:
for images,labels in train_loader:
  preds=model(images)
  print(preds.shape)
  break

torch.Size([128, 10])


In [53]:
# !pip install torchviz
# !pip install onnx


In [46]:
from torchviz import make_dot

make_dot(preds, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

'rnn_torchviz.png'

In [54]:
import onnx
for images,labels in val_loader:
  torch.onnx.export(model, images, 'torch_cnn_cfiar.onnx', input_names=["features"], output_names=["logits"])
  break


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

